In [ ]:
http://bigdataexaminer.com/data-science/understanding-bias-and-variance-using-python/

# Luther Problem Statement
Source: https://github.com/thisismetis/nyc16_ds7/tree/master/projects/02-luther

Back story:

Using information we scrape from the web, can we build linear regression models from which we can learn about the movie industry?

Data:

acquisition: web scraping
storage: flat files
sources: Box Office Mojo, The Numbers, any other publicly available information
Skills:

basics of the web (requests, HTML, CSS, JavaScript)
web scraping
numpy and pandas
statsmodels, scikit-learn
Analysis:

linear regression
Deliverable/communication:

organized project repository
slide presentation
visual and oral communication in presentations
write-up of process and results
Design:

iterative design process
scoping
"MVP"s and building outward
More information:

We'll learn about web scraping using two popular tools - BeautifulSoup and Selenium. You'll have to know the very basics of HTML. We'll also be evolving the way we use IPython notebooks—during this project we'll begin to use the notebook as a development scratchpad, where we test things out through interactive scripting, but then solidify our work in python modules with reusable functions and classes.

We'll practice using linear regression. We'll have a first taste of feature selection, this time based on our intuition and some trial and error, and we'll build and refine our models.

We'll work in groups for brainstorming and design, and code sharing will be highly encouraged, but the final projects will be individual.

This project will really give you the freedom to challenge yourself, no matter your skill level. Find your boundaries, meet them, and push them a little further.

We are very excited to see what you will learn and do for Project Luther!

# Luther Project Summary Statement
Here I describe where I got my data. What my hypothesis is all about
Introduction explaining the study and source of the day. And what terms mean

# Background Information on Variables

http://research.microsoft.com/pubs/163535/FacebookPersonality_michal_29_04_12.pdf

# Import Data

In [54]:
import pandas as pd
import statsmodels.api as sm
from sklearn import preprocessing as pp

In [6]:
!ls mypersonality_final/

celli-al_wcpr13.pdf     mypersonality_final.csv readme.txt


In [7]:
df_raw = pd.read_csv('mypersonality_final/mypersonality_final.csv')

In [8]:
my_dict = {"Joe":25, "Mary":23, "Bob":42}

In [14]:
df_raw.shape

(9917, 20)

In [11]:
df_raw.head(3)

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [41]:
df_grouped = df_raw.groupby(by = "#AUTHID", as_index = False).first()

In [44]:
df_grouped.describe()

,sEXT,sNEU,sAGR,sCON,sOPN,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
count,250.000000,250.000000,250.000000,250.000000,250.00000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,3.292120,2.627240,3.600320,3.522920,4.07856,408.012000,128936.801920,93.980960,0.033760,131342.232000,0.488320,0.138600
std,0.861368,0.776835,0.670758,0.740211,0.57506,316.636458,204807.609367,7.080237,0.036009,207014.930192,0.015587,0.113439
min,1.330000,1.250000,1.650000,1.450000,2.25000,24.000000,185.710000,31.210000,0.000000,241.000000,0.320000,0.000000
25%,2.662500,2.000000,3.150000,3.000000,3.75000,175.000000,14062.350000,93.260000,0.010000,14576.250000,0.490000,0.060000
50%,3.300000,2.590000,3.660000,3.500000,4.15000,315.000000,46636.100000,96.020000,0.020000,48358.000000,0.490000,0.110000
75%,4.000000,3.150000,4.100000,4.050000,4.50000,556.500000,145483.000000,97.807500,0.040000,152115.750000,0.500000,0.180000
max,5.000000,4.750000,5.000000,5.000000,5.00000,1596.000000,1251780.000000,99.820000,0.400000,1263790.000000,0.500000,0.630000


In [67]:
x_extroversion = df_grouped["sEXT"]
x_neuroticism = df_grouped["sNEU"]
x_agreeableness = df_grouped["sAGR"]
x_conscientiousness = df_grouped["sCON"]
x_openness = df_grouped["sOPN"]

y_betweenness = df_grouped["BETWEENNESS"]
y_nbetweenness = df_grouped["NBETWEENNESS"]

network_size = df_grouped["NETWORKSIZE"]


In [72]:
variable_list = [x_extroversion,x_neuroticism,x_agreeableness,x_conscientiousness,x_openness]

In [74]:
for var in variable_list:
    print var.shape

(250,)
(250,)
(250,)
(250,)
(250,)


In [79]:
x_individuals = pd.DataFrame([x_extroversion,x_neuroticism,x_agreeableness,x_conscientiousness,x_openness]).T

In [80]:
x_individuals.shape

(250, 5)

In [81]:
x_individuals.head(10)

,sEXT,sNEU,sAGR,sCON,sOPN
0,1.90,4.15,3.15,4.65,3.45
1,2.05,4.00,2.20,2.20,3.75
2,3.84,3.55,2.85,4.32,3.45
3,3.20,3.60,3.85,4.35,4.80
4,2.85,2.35,3.35,4.70,3.35
5,3.00,3.15,2.95,3.55,4.80
6,3.35,2.30,4.10,2.40,4.50
7,4.33,2.17,3.17,4.17,4.83
8,2.80,1.60,3.65,3.25,4.15
9,2.85,3.20,2.85,4.10,4.70


In [228]:
X = PolynomialFeatures_labeled(x_individuals,2)
X.head()

,Constant Term,sEXT^1,sNEU^1,sAGR^1,sCON^1,sOPN^1,sEXT^2,sEXT^1 x sNEU^1,sEXT^1 x sAGR^1,sEXT^1 x sCON^1,...,sNEU^2,sNEU^1 x sAGR^1,sNEU^1 x sCON^1,sNEU^1 x sOPN^1,sAGR^2,sAGR^1 x sCON^1,sAGR^1 x sOPN^1,sCON^2,sCON^1 x sOPN^1,sOPN^2
0,1,1.90,4.15,3.15,4.65,3.45,3.6100,7.8850,5.9850,8.8350,...,17.2225,13.0725,19.2975,14.3175,9.9225,14.6475,10.8675,21.6225,16.0425,11.9025
1,1,2.05,4.00,2.20,2.20,3.75,4.2025,8.2000,4.5100,4.5100,...,16.0000,8.8000,8.8000,15.0000,4.8400,4.8400,8.2500,4.8400,8.2500,14.0625
2,1,3.84,3.55,2.85,4.32,3.45,14.7456,13.6320,10.9440,16.5888,...,12.6025,10.1175,15.3360,12.2475,8.1225,12.3120,9.8325,18.6624,14.9040,11.9025
3,1,3.20,3.60,3.85,4.35,4.80,10.2400,11.5200,12.3200,13.9200,...,12.9600,13.8600,15.6600,17.2800,14.8225,16.7475,18.4800,18.9225,20.8800,23.0400
4,1,2.85,2.35,3.35,4.70,3.35,8.1225,6.6975,9.5475,13.3950,...,5.5225,7.8725,11.0450,7.8725,11.2225,15.7450,11.2225,22.0900,15.7450,11.2225


# Clean Data as Necesesary

In [229]:
df_raw.describe()

,sEXT,sNEU,sAGR,sCON,sOPN,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
count,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9916.000000
mean,3.354760,2.609453,3.616643,3.474201,4.130386,429.377120,135425.315359,94.665170,3.154012,137642.476201,0.489920,0.128821
std,0.857578,0.760248,0.682485,0.737215,0.585672,428.760382,199433.803497,5.506696,311.073343,201392.066555,0.011908,0.106063
min,1.330000,1.250000,1.650000,1.450000,2.250000,24.000000,93.250000,0.040000,0.000000,0.490000,0.180000,0.000000
25%,2.710000,2.000000,3.140000,3.000000,3.750000,196.000000,16902.200000,93.770000,0.010000,17982.000000,0.490000,0.060000
50%,3.400000,2.600000,3.650000,3.400000,4.250000,317.000000,47166.900000,96.440000,0.020000,48683.000000,0.490000,0.090000
75%,4.000000,3.050000,4.150000,4.000000,4.550000,633.000000,196606.000000,97.880000,0.030000,198186.000000,0.500000,0.170000
max,5.000000,4.750000,5.000000,5.000000,5.000000,29724.900000,1251780.000000,99.820000,30978.000000,1263790.000000,0.500000,0.630000


In [230]:
df_raw.shape

(9917, 20)

In [231]:
# G
df_raw.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [232]:
df_raw.groupby(#AUTHID, idex = false)

SyntaxError: unexpected EOF while parsing (<ipython-input-232-14a2a09d8e74>, line 1)

# Summary Statistics & Descriptive Visualizations (Pandas and Matplotlib, etc)

In [233]:
### Plot network size

### Printing the covaraiance matrix

In [238]:
np.cov(np.vstack([x_agreeableness,x_conscientiousness,x_extroversion,x_neuroticism,x_openness]))


array([[ 0.44991636,  0.04948019,  0.11868526, -0.21926337,  0.06930046],
       [ 0.04948019,  0.54791232,  0.12810945, -0.13474452,  0.03516567],
       [ 0.11868526,  0.12810945,  0.74195493, -0.25243188,  0.10056692],
       [-0.21926337, -0.13474452, -0.25243188,  0.60347227, -0.06037226],
       [ 0.06930046,  0.03516567,  0.10056692, -0.06037226,  0.3306935 ]])

### Perform Scatter plots (y vs. Xi) for each of the X's -- this suggest relationship

### Check features for normality by using a density plot - plot normal on top of it. If it's not normal, then do some transformations on the features until they get to be normal

# Testing (Regression)

### Step 1a: (via intuition) Generate different models and provide explanations of why (avoid products, okay for squares and logs)

### Step 1b: Algorithmly - go throuhg all the options and create them.

### Step 2: Separate Data into 3 groupings: training, cv, and 
- Create seed

In [241]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y_betweenness, test_size=0.4, random_state=0)

### Step 3: Using the Training set, determine the best coefficients for each model... (Fitting a model -- sm.fit()))

In [242]:
import statsmodels.api as sm
model = sm.OLS(y_train,X_train)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            BETWEENNESS   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     1.453
Date:                Mon, 18 Apr 2016   Prob (F-statistic):              0.110
Time:                        23:23:05   Log-Likelihood:                -2015.1
No. Observations:                 150   AIC:                             4072.
Df Residuals:                     129   BIC:                             4135.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Constant Term    1.408e+06   1.75e+06      0.804      0.423     -2.06e+06  4.87e+06
sEXT^1           8.653e+04    2.9e+05      0.299      0.766     -4.87e+05   6.6e+05
sNEU^1           1.893e+05   3.39e+05      0.558      0.578     -4.82e+05  8.61e+05
sAGR^1          -1.781e+05   4.16e+05     -0.428      0.669        -1e+06  6.45e+05
sCON^1          -2.687e+05   2.85e+05     -0.942      0.348     -8.33e+05  2.96e+05
sOPN^1          -5.772e+05   3.78e+05     -1.527      0.129     -1.33e+06  1.71e+05
sEXT^2           8503.1872   2.32e+04      0.366      0.715     -3.74e+04  5.44e+04
sEXT^1 x sNEU^1 -2.545e+04   2.68e+04     -0.949      0.344     -7.85e+04  2.76e+04
sEXT^1 x sAGR^1 -1.232e+04   3.89e+04     -0.317      0.752     -8.93e+04  6.46e+04
sEXT^1 x sCON^1  2.247e+04   2.72e+04      0.825      0.411     -3.14e+04  7.64e+04
sEXT^1 x sOPN^1 -1.454e+04   2.99e+04     -0.487      0.627     -7.36e+04  4.45e+04
sNEU^2          -9507.1485   2.51e+04     -0.379      0.705     -5.91e+04  4.01e+04
sNEU^1 x sAGR^1 -4752.3801   3.86e+04     -0.123      0.902     -8.12e+04  7.17e+04
sNEU^1 x sCON^1  2911.7070   2.94e+04      0.099      0.921     -5.53e+04  6.12e+04
sNEU^1 x sOPN^1 -1.346e+04   3.72e+04     -0.362      0.718      -8.7e+04  6.01e+04
sAGR^2            1.84e+04   3.58e+04      0.513      0.609     -5.25e+04  8.93e+04
sAGR^1 x sCON^1   1.12e+04   4.01e+04      0.279      0.781     -6.82e+04  9.06e+04
sAGR^1 x sOPN^1  2.424e+04   4.51e+04      0.537      0.592      -6.5e+04  1.14e+05
sCON^2          -7905.6155   2.36e+04     -0.336      0.738     -5.45e+04  3.87e+04
sCON^1 x sOPN^1  5.759e+04   3.14e+04      1.833      0.069     -4580.092   1.2e+05
sOPN^2           4.862e+04   3.54e+04      1.372      0.173     -2.15e+04  1.19e+05
==============================================================================
Omnibus:                      101.524   Durbin-Watson:                   1.658
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              661.249
Skew:                           2.476   Prob(JB):                    2.58e-144
Kurtosis:                      12.015   Cond. No.                     6.08e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.08e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Step 4: Against the CV set, compare the cost functions for each of the models and choose which model is the best (predict... and then get the difference) + ALso compare the adjusted R squared for all of them

### Step 5: Once the model is chosen, retrain that model using the combination of CV + Training

### Step 6: Test your final model against your test set.

In [ ]:
Diagnostic Tool #1:

https://theclevermachine.files.wordpress.com/2013/04/bias-variance-train-test-error.png

In [ ]:
Diagnostic Tool #2
### Step 7 - If you increase the size of your training set (from 0 to the full amount)
http://www.bigdataexaminer.com/wp-content/uploads/2014/11/code-9.png

# Analysis & Interpretations

# Next Steps

In [ ]:
Add sentiment analysis stuff
# Create Repository called luther (?)

# Extract out #auth & status updates

# Generate some "word usage" variable per person (collapse all statuses into a single metric per person)

# Add that metric to your model
# See if that's representative for the centrality metrics.

In [206]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp

a = np.ones(3)
b = np.ones(3) * 2
c = np.ones(3) * 3

input_df = pd.DataFrame([a,b,c])
input_df = input_df.T
print blah_df
blah_df.columns=['a', 'b', 'c']

   0  1  2
0  1  2  3
1  1  2  3
2  1  2  3


In [207]:
blah_df

,a,b,c
0,1,2,3
1,1,2,3
2,1,2,3


In [135]:
poly = pp.PolynomialFeatures(2)
output_nparray = poly.fit_transform(blah_df)

In [130]:
input_df = pd.DataFrame([a,b,c], columns=['a', 'b', 'c'])
feature_names = input_df.columns

In [131]:
feature_names = input_df.columns

In [133]:
print type(output_df)

<type 'numpy.ndarray'>


In [ ]:
poly = pp.PolynomialFeatures(2)
poly.fit_transform(x_individuals)
poly.powers_

0


In [209]:
#??? What's the best way to write this function? WHat are the ideal inputs and outputs?


def PolynomialFeatures_labeled(input_df,power):
    '''Basically
    
    Input = the input to poly.fit_transform
    Output_df = the thing which is the result of poly.fit_transform(blah_df)
    It basically outputs output_nparray as a labeled dataframe
    '''
    poly = pp.PolynomialFeatures(power)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_
    
    
    input_feature_names = list(input_df.columns)
    #print input_feature_names
    target_feature_names = ["Constant Term"]
    #print output_nparray
    #print target_feature_names
    #print powers_nparray[1:]
    
    # Iterate across them
    # (Powers) If it's 0, move on. 1 leave as is.
    # (Multipliers)
    for feature_distillation in powers_nparray[1:]:
        #print feature_distillation
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):

            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s^%d"%(variable,power)
                #print "intermediary: ", intermediary_label
                #print len(final_label)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                    #print "final: ", final_label
                else:
                    final_label = final_label + " x " + intermediary_label
                    #print "final: ", final_label
        target_feature_names.append(final_label)
#    print target_feature_names
    import pandas as pd
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df
            
blah = PolynomialFeatures_labeled(blah_df,3)
print blah

   Constant Term  a^1  b^1  c^1  a^2  a^1 x b^1  a^1 x c^1  b^2  b^1 x c^1  \
0              1    1    2    3    1          2          3    4          6   
1              1    1    2    3    1          2          3    4          6   
2              1    1    2    3    1          2          3    4          6   

   c^2  a^3  a^2 x b^1  a^2 x c^1  a^1 x b^2  a^1 x b^1 x c^1  a^1 x c^2  b^3  \
0    9    1          2          3          4                6          9    8   
1    9    1          2          3          4                6          9    8   
2    9    1          2          3          4                6          9    8   

   b^2 x c^1  b^1 x c^2  c^3  
0         12         18   27  
1         12         18   27  
2         12         18   27  


In [214]:
#??? What's the best way to write this function? WHat are the ideal inputs and outputs?


def PolynomialFeatures_labeled(input_df,power):
    '''Basically this is a cover for the sklearn preprocessing function. 
    The problem with that function is if you give it a labeled dataframe, it ouputs an unlabeled dataframe with potentially
    a whole bunch of unlabeled columns. 
    
    Inputs:
    input_df = Your labeled pandas dataframe (list of x's not raised to any power) 
    power = what order polynomial you want variables up to. (use the same power as you want entered into pp.PolynomialFeatures(power) directly)
    
    Ouput:
    Output: This function relies on the powers_ matrix which is one of the preprocessing function's outputs to create logical labels and 
    outputs a labeled pandas dataframe   
    '''
    poly = pp.PolynomialFeatures(power)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_
    
    input_feature_names = list(input_df.columns)
    target_feature_names = ["Constant Term"]
    for feature_distillation in powers_nparray[1:]:
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):
            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s^%d" % (variable,power)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                else:
                    final_label = final_label + " x " + intermediary_label
        target_feature_names.append(final_label)
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df


print blah_df

output_df = PolynomialFeatures_labeled(blah_df,3)
print type(output_df)
print output_df

   a  b  c
0  1  2  3
1  1  2  3
2  1  2  3
<class 'pandas.core.frame.DataFrame'>
   Constant Term  a^1  b^1  c^1  a^2  a^1 x b^1  a^1 x c^1  b^2  b^1 x c^1  \
0              1    1    2    3    1          2          3    4          6   
1              1    1    2    3    1          2          3    4          6   
2              1    1    2    3    1          2          3    4          6   

   c^2  a^3  a^2 x b^1  a^2 x c^1  a^1 x b^2  a^1 x b^1 x c^1  a^1 x c^2  b^3  \
0    9    1          2          3          4                6          9    8   
1    9    1          2          3          4                6          9    8   
2    9    1          2          3          4                6          9    8   

   b^2 x c^1  b^1 x c^2  c^3  
0         12         18   27  
1         12         18   27  
2         12         18   27  


In [149]:
PolynomialFeatures_labeled(input_df,2)

Index([u'a', u'b', u'c'], dtype='object')
[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  2.  2.  2.  4.  4.  4.  4.  4.  4.]
 [ 1.  3.  3.  3.  9.  9.  9.  9.  9.  9.]]
['Constant Term']
[[0 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [2 0 0]
 [1 1 0]
 [1 0 1]
 [0 2 0]
 [0 1 1]
 [0 0 2]]
